In [95]:
import numpy as np
from mpi4py import MPI

from dolfinx.mesh import CellType, create_box, locate_entities_boundary
from dolfinx.fem import (Constant, DirichletBC, Function, LinearProblem, FunctionSpace, VectorFunctionSpace, 
                         locate_dofs_topological)
from ufl import (Identity, SpatialCoordinate, TestFunction, TrialFunction,
                 as_vector, dx, grad, inner, sym, tr)
import dolfinx

In [124]:
# Scaled variable
L = 1 #m
W = 0.2 #m
mu = 1
rho = 2500 #kg/m^3
delta = W/L
gamma = 0.4*delta**2
# Elasticity parameters

E = 70e9
nu = 0.2
mu = E / (2.0 * (1.0 + nu))
lambda_ = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))
g = 9.8 #acceleration due to gravity
omega=1
mesh = create_box(MPI.COMM_WORLD, [np.array([0,0,0]), np.array([L, W, W])],
                  [20,6,6], cell_type=CellType.hexahedron)
V = VectorFunctionSpace(mesh, ("CG", 1))

In [126]:

def sigma(v):
    return 2.0 * mu * sym(grad(v)) + lambda_ * tr(sym(grad(v))) * Identity(
        len(v))


# Create function space
V = VectorFunctionSpace(mesh, ("Lagrange", 1))

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
a = inner(sigma(u), grad(v)) * dx
L = inner(f, v) * dx

TypeError: '<' not supported between instances of 'Mesh' and 'Mesh'

In [60]:
def cell_vol(mesh,g_indicies,c):
    return np.prod(np.abs(mesh.geometry.x[g_indices[c][0]] - mesh.geometry.x[g_indices[c]][7]))**(1/3)

In [103]:
x = SpatialCoordinate(mesh)
f = as_vector((rho * omega**2 * x[0], rho * omega**2 * x[1], 0.0))


In [73]:
# determine mass for the individual units and apply that as a local force in the -z direction
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
g_indices = dolfinx.cpp.mesh.entities_to_geometry(
    mesh, mesh.topology.dim, np.arange(num_cells, dtype=np.int32), False)
mass_values = []
for c in range(num_cells):
    mass_values.append(rho*cell_vol(mesh,g_indices,c))
mass_values=np.array(mass_values)

In [65]:
def clamped_boundary(x):
    return np.isclose(x[0], 0)

fdim = mesh.topology.dim - 1
boundary_facets = locate_entities_boundary(mesh, fdim, clamped_boundary)

u_D = Function(V)
u_D.x.array[:] = 0
bc = DirichletBC(u_D, locate_dofs_topological(V, fdim, boundary_facets))

In [66]:
from petsc4py.PETSc import ScalarType
T = Constant(mesh, ScalarType((0, 0, 0)))

In [67]:
import ufl
ds = ufl.Measure("ds", domain=mesh) # displacement

In [93]:
# build vector for fenics
ff = Constant(mesh,ScalarType((np.zeros_like(mass_values),np.zeros_like(mass_values),mass_values)))


(720,)

In [68]:
def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

# set up elastic equations
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# define the input force
f = Constant(mesh, ScalarType((0, 0, -rho*g)))
# set up the solution
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
# ???? apply the solution displacement I guess?
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

In [69]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [70]:
import pyvista
from dolfinx.plot import create_vtk_topology
# Start xvfb for rendering plots
pyvista.start_xvfb(wait=0.05)

# Create plotter and pyvista grid
p = pyvista.Plotter(title="Deflection", window_size=[800, 800])
topology, cell_types = create_vtk_topology(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.compute_point_values().real 
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped)
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
    figure_as_array = p.screenshot("deflection.png")

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=800)